## Response API방식

### 1.라이브러리 가져오고 api key를 환경 변수에서 가져오기

In [ ]:
import os
import base64
from io import BytesIO

from PIL import Image
# from dotenv import load_dotenv
from openai import OpenAI

# load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# client는 보통 전역 1회 생성 권장
client = OpenAI(api_key=api_key)

### 2.이미지를 문자열로 인코딩하는 함수 정의하기

In [9]:
# 이미지를 base64 문자열로 인코딩하는 함수
def encode_image(img: Image.Image, max_side: int = 512) -> str:
    w, h = img.size
    scale = min(1.0, max_side / max(w, h))
    if scale < 1.0:
        img = img.resize((int(w * scale), int(h * scale)))

    buf = BytesIO()
    img.convert("RGB").save(buf, format="JPEG", quality=90)
    return base64.b64encode(buf.getvalue()).decode("utf-8")

### 3.모델이 이미지 분류 요청 함수 정의하기

In [10]:
# GPT 모델에게 이미지와 프롬프트를 보내 결과를 받아오는 함수
def classify_image(prompt: str, img: Image.Image, model: str = "gpt-4o") -> str:
    b64 = encode_image(img)
    data_uri = f"data:image/jpeg;base64,{b64}"

    resp = client.responses.create(
        model=model,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": prompt},
                    {"type": "input_image", "image_url": data_uri},
                ],
            }
        ],
        temperature=0,
    )

    return resp.output_text

### 4.프롬프트 선언하고 이미지 분류 실행하기

In [11]:
# GPT에게 보낼 프롬프트 정의
prompt = """
영상을 보고 다음 보기 내용이 포함되면 1, 포함되지 않으면 0으로 분류해줘.
보기 = [건축물, 바다, 산]
JSON format으로 키는 'building', 'sea', 'mountain'으로 하고 각각 건축물, 바다, 산에 대응되도록 출력해줘.
자연 이외의 건축물이 조금이라도 존재하면 'building'을 1로, 물이 조금이라도 존재하면 'sea'을 1로, 산이 조금이라도 보이면 'mountain'을 1로 설정해줘.
markdown format은 포함하지 말아줘.
"""

img = Image.open('imgs_classification/01.jpg')  # 이미지 열기
response = classify_image(prompt, img)     # GPT로부터 분류 결과 받기
print(response)  # 결과 출력



{
    "building": 1,
    "sea": 0,
    "mountain": 1
}


In [1]:
import sys
print(sys.executable)

c:\Users\chang\anaconda3\envs\llm_img\python.exe


In [ ]:
%pip install streamlit

In [4]:
# -0) 라이브러리 추가하기 : streamlit
import streamlit as st

In [12]:
# -1) model 선택하기 : st.sidebar / st.selectbox
with st.sidebar:
    st.header("설정")
    
    # API Key가 환경변수에 없으면 입력받기 (선택사항)
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        api_key = st.text_input("OpenAI API Key를 입력하세요", type="password")
    
    selected_model = st.selectbox(
        "모델 선택", 
        ["gpt-4o", "gpt-4-turbo", "gpt-4o-mini"],
        index=0
    )

2026-01-14 10:51:27.765 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.766 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.779 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:51:27.780 Session state does not function when running a script without `streamlit run`
2026-01-14 10:51

In [15]:
# -2) prompt 작성하기 : st.text_area

default_prompt = """
영상을 보고 다음 보기 내용이 포함되면 1, 포함되지 않으면 0으로 분류해줘.
보기 = [건축물, 바다, 산]
JSON format으로 키는 'building', 'sea', 'mountain'으로 하고 각각 건축물, 바다, 산에 대응되도록 출력해줘.
자연 이외의 건축물이 조금이라도 존재하면 'building'을 1로, 물이 조금이라도 존재하면 'sea'을 1로, 산이 조금이라도 보이면 'mountain'을 1로 설정해줘.
markdown format은 포함하지 말아줘.
"""

st.subheader("1. 프롬프트 작성")
user_prompt = st.text_area("이미지 분석을 위한 프롬프트를 입력하세요:", value=default_prompt.strip(), height=200)

2026-01-14 10:52:41.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.338 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.340 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.342 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.345 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.354 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:41.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [16]:
# -3) 이미지 업로드하기 : st.file_uploader

st.subheader("2. 이미지 업로드")
uploaded_file = st.file_uploader("분석할 이미지를 선택하세요 (jpg, png, jpeg)", type=["jpg", "png", "jpeg"])

2026-01-14 10:52:53.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.789 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.791 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 10:52:53.793 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [26]:
# -4) 업로드한 이미지 보여주기 : st.image

if uploaded_file is not None:
    # PIL Image로 변환
    image = Image.open(uploaded_file)
    
    # 화면을 2분할하여 왼쪽엔 이미지, 오른쪽엔 결과를 보여줌
    col1, col2 = st.columns([1, 1])
    
    with col1:
        st.image(image, caption='업로드된 이미지', use_column_width=True)

# -5) 분류 실행하기 : st.button /st.spinner
    with col2:
        st.subheader("3. 분석 결과")
        if st.button("분류 실행하기", use_container_width=True):
            with st.spinner("GPT가 이미지를 분석 중입니다..."):
                try:
                    # 함수 호출
                    result = classify_image(client, user_prompt, image, selected_model)
                    
                    # -6) 결과 출력하기 : st.write / st.code
                    st.success("분석 완료!")
                    st.write("결과 (JSON):")
                    st.code(result, language='json')
                    
                except Exception as e:
                    st.error(f"오류가 발생했습니다: {e}")